# 3. Welcome To LangChain - Recap

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

chef_prompt = ChatPromptTemplate.from_messages([
    ('system', "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ('human', "I want to cook {cuisine} food.")
])

chef_chain = chef_prompt | chat

veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ('human', "{recipe}")
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe" : chef_chain} | veg_chain

final_chain.invoke({
    "cuisine" : "Italian",
})

Great choice! Italian cuisine is known for its delicious flavors and simple ingredients. Let's start with a classic recipe for Spaghetti Aglio e Olio, a traditional Italian pasta dish.

Ingredients:
- 1/2 pound spaghetti
- 4 cloves of garlic, thinly sliced
- 1/4 cup extra virgin olive oil
- 1/2 teaspoon red pepper flakes
- Salt and pepper to taste
- Fresh parsley, chopped (for garnish)
- Grated Parmesan cheese (optional)

Instructions:
1. Cook the spaghetti in a large pot of salted boiling water according to the package instructions until al dente. Reserve 1 cup of pasta water before draining the spaghetti.

2. In a large skillet, heat the olive oil over medium heat. Add the sliced garlic and red pepper flakes, and sauté until the garlic is golden brown and fragrant, about 1-2 minutes. Be careful not to burn the garlic.

3. Add the cooked spaghetti to the skillet along with a splash of the reserved pasta water. Toss the spaghetti in the garlic-infused oil until well coated. If the past

AIMessageChunk(content="For the Spaghetti Aglio e Olio recipe, here are some vegetarian alternatives for the ingredients:\n\n1. **Spaghetti**: You can use whole wheat spaghetti or gluten-free spaghetti if you prefer.\n   \n2. **Extra Virgin Olive Oil**: No need to change this ingredient, as it's already vegetarian.\n\n3. **Garlic**: Garlic is a key ingredient in this recipe and doesn't need to be substituted.\n\n4. **Red Pepper Flakes**: Red pepper flakes add a nice kick to the dish. If you prefer a milder heat, you can reduce the amount or omit them altogether.\n\n5. **Salt and Pepper**: These can remain the same.\n\n6. **Fresh Parsley**: Fresh parsley is a great herb to garnish the dish. You can also use fresh basil or chives for a different flavor profile.\n\n7. **Grated Parmesan Cheese**: You can use vegetarian-friendly Parmesan cheese or nutritional yeast as a substitute for the traditional Parmesan cheese. Nutritional yeast has a cheesy flavor and adds a nice umami touch to the d

# LCEL Chains Challenge

In [2]:
# 필요한 라이브러리 import
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

# ChatOpenAI model setting
chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

# chain that is specialized in writing Haikus about programming languages
haiku_prompt = ChatPromptTemplate.from_messages([
    ('system', "프로그래밍 언어에 대한 하이쿠를 작성하는 시인입니다."),
    ('human', "{programming_language}에 대한 하이쿠를 작성하세요.")
])
haiku_chain = haiku_prompt | chat

# chain that is specialized in explaining Haikus
explanation_prompt = ChatPromptTemplate.from_messages([
    ("system", "프로그래밍 언어에 대한 하이쿠의 더 깊은 의미를 설명하는 전문가입니다."),
    ('human', "이 하이쿠를 설명하세요: {haiku}")
])
explanation_chain = explanation_prompt | chat

def handle_streaming_response(streaming_response):
    accumulated_text = ''
    for chunk in streaming_response:
        if isinstance(chunk, dict) and 'text' in chunk:
            accumulated_text += chunk['text']
        elif isinstance(chunk, tuple) and chunk[0] == 'content':
            accumulated_text += chunk[1]
        else:
            print(f"Unexpected chunk format or type: {chunk}")
    return accumulated_text

def connect_chains_and_provide_explanation(programming_language):
    try:
        haiku_response = haiku_chain.invoke({"programming_language": programming_language})
        haiku_text = handle_streaming_response(haiku_response)

        explanation_response = explanation_chain.invoke({"haiku": haiku_text})
        explanation_text = handle_streaming_response(explanation_response)

        return {"haiku": haiku_text, "explanation": explanation_text}
    except Exception as e:
        print(f"Chain processing error: {e}")
        return {}

# Python에 대한 Haikus 와 설명 출력
result = connect_chains_and_provide_explanation("Python")
print(result)


파이썬 코드,
간결하고 강력한,
프로그래밍 덕후.Unexpected chunk format or type: ('additional_kwargs', {})
Unexpected chunk format or type: ('type', 'AIMessageChunk')
Unexpected chunk format or type: ('example', False)
이 하이쿠는 파이썬 프로그래밍 언어에 대한 감탄과 애정을 담고 있습니다. "파이썬 코드"는 파이썬 언어를 가리키며, "간결하고 강력한"은 파이썬의 간결한 문법과 풍부한 기능을 나타냅니다. 마지막 줄인 "프로그래밍 덕후"는 파이썬을 좋아하고 열정적으로 사용하는 프로그래머들을 지칭합니다. 이 하이쿠는 파이썬의 매력과 사용자들의 열정을 표현하고 있습니다.Unexpected chunk format or type: ('additional_kwargs', {})
Unexpected chunk format or type: ('type', 'AIMessageChunk')
Unexpected chunk format or type: ('example', False)
{'haiku': '파이썬 코드,\n간결하고 강력한,\n프로그래밍 덕후.', 'explanation': '이 하이쿠는 파이썬 프로그래밍 언어에 대한 감탄과 애정을 담고 있습니다. "파이썬 코드"는 파이썬 언어를 가리키며, "간결하고 강력한"은 파이썬의 간결한 문법과 풍부한 기능을 나타냅니다. 마지막 줄인 "프로그래밍 덕후"는 파이썬을 좋아하고 열정적으로 사용하는 프로그래머들을 지칭합니다. 이 하이쿠는 파이썬의 매력과 사용자들의 열정을 표현하고 있습니다.'}
